In [1]:
#import python and pytorch libraries
from __future__ import absolute_import
from __future__ import print_function

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import codecs
import os
import sys
import numpy as np
import logging
import tempfile
import shutil
import pickle
import platform
import json
from datetime import datetime
import random
from sklearn import metrics
import matplotlib.pyplot as plt
import itertools
import collections
from collections import defaultdict

In [3]:
# !pip install import_ipynb

In [4]:
import import_ipynb
from mimic_utils_text import InHospitalMortalityReader, Discretizer, Normalizer, read_chunk

importing Jupyter notebook from mimic_utils_text.ipynb


c:\Users\DAIMA Researcher\anaconda3\Lib\site-packages\nbformat\__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


# Pytorch Dataset

In [5]:
# pad missing values in the nested lists with 0s
def pad_missing_value_with_zero(data):
    a1 = np.zeros((len(data), max([len(k) for k in data]), 35))
    for ctr,k in enumerate(data):
        a1[ctr,:len(k),:] = k
    return a1

In [6]:
# pytroch class for reading data into batches
class MIMICDataset(Dataset):
    """
       Loads time series data into memory from a text file,
       split by newlines. 
    """
    def __init__(self, reader, target_repl=False, batch_labels=False):
        self.data = []
        self.y  = []
        N = reader.get_number_of_examples()
        print(N)
        # read data form cvs files
        ret = read_chunk(reader, N)
        # read into memory structured data X and labels y
        data = ret["X"]
        ts = ret["t"]
        labels = ret["y"]
        names = ret["name"]
        
        # pad missing values in the list of arrays with 0s
        data = pad_missing_value_with_zero(data)
        
        self.data = np.array(data, dtype=np.float32) 
        self.T = self.data.shape[1]

        if batch_labels:
            self.y = np.array([[l] for l in labels], dtype=np.float32)
        else:
            self.y = np.array(labels, dtype=np.float32)
        if target_repl:
            self.y = self._extend_labels(self.y)

    def _extend_labels(self, labels):
        # (B,)
        labels = labels.repeat(self.T, axis=1)  # (B, T)
        return labels

    def __len__(self):
        # overide len to get number of instances
        return len(self.data)

    def __getitem__(self, idx):
        # get features (physiological variables x) and label for a given instance index
        return self.data[idx], self.y[idx]

# Evaluation metrics
The main measure used are accuracy, ROC AUC and PR AUC.

In [7]:
# eval metrics
def print_metrics_binary(y_true, predictions, logging, verbose=1):
    predictions = np.array(predictions)
    if len(predictions.shape) == 1:
        predictions = np.stack([1 - predictions, predictions]).transpose((1, 0))
    cf = metrics.confusion_matrix(y_true, predictions.argmax(axis=1))
    if verbose:
        logging.info("confusion matrix:")
        logging.info(cf)
    cf = cf.astype(np.float32)

    acc = (cf[0][0] + cf[1][1]) / np.sum(cf)
    prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
    prec1 = cf[1][1] / (cf[1][1] + cf[0][1])
    rec0 = cf[0][0] / (cf[0][0] + cf[0][1])
    rec1 = cf[1][1] / (cf[1][1] + cf[1][0])
    auroc = metrics.roc_auc_score(y_true, predictions[:, 1])

    (precisions, recalls, thresholds) = metrics.precision_recall_curve(y_true, predictions[:, 1])
    auprc = metrics.auc(recalls, precisions)
    minpse = np.max([min(x, y) for (x, y) in zip(precisions, recalls)])

    if verbose:
        logging.info("accuracy = {0:.3f}".format(acc))
        logging.info("precision class 0 = {0:.3f}".format(prec0))
        logging.info("precision class 1 = {0:.3f}".format(prec1))
        logging.info("recall class 0 = {0:.3f}".format(rec0))
        logging.info("recall class 1 = {0:.3f}".format(rec1))
        logging.info("AUC of ROC = {0:.3f}".format(auroc))
        logging.info("AUC of PRC = {0:.3f}".format(auprc))
       

    return {"acc": acc,
            "prec0": prec0,
            "prec1": prec1,
            "rec0": rec0,
            "rec1": rec1,
            "auroc": auroc,
            "auprc": auprc}

In [8]:
def performance_with_cutoff (y_test, pred_probabilities, verbose=1):
    # performance
    fpr, tpr, thresholds = metrics.roc_curve(y_test, pred_probabilities)
    # compute roc auc
    roc_auc = metrics.roc_auc_score(y_test, pred_probabilities, average = 'micro')
    # compute Precision_Recall curves
    precision, recall, _ = metrics.precision_recall_curve(y_test, pred_probabilities)
    # compute PR_AUC
    pr_auc = metrics.auc(recall, precision)
       
    # compute confusion matrix
    optimal_cut_off = round(thresholds[np.argmax(tpr - fpr)],4)
    a = np.where(pred_probabilities > optimal_cut_off, 1, 0)
    brier = round(metrics.brier_score_loss(y_test, pred_probabilities, sample_weight=None, pos_label=None),3)
    predictions = np.where(pred_probabilities > optimal_cut_off, 1, 0)  
    matrix = metrics.confusion_matrix(y_test, a, labels=None, normalize=None)
    
    if verbose:
        print("confusion matrix after tuning with cutoff")
        print('Cut off: ' + str(optimal_cut_off))
        print(matrix)
        print("accuracy with cutoff = {0:.3f}".format(metrics.accuracy_score(y_test, predictions)))

# Training
Here we define the model, loss and training loop. The model is an LSTM. 

In [9]:
#model
class LSTMClassifier(nn.Module):

    def __init__(self, tag_size, hidden_size, feat_size, emb_size,
                 bidirectional=False, dropout=0.2, aggregation_type='last_state'):
        """
        constructor, here we define the hidden layers for our architecture       
        """
        super().__init__()
        # define if the rnn will be bidirectional
        self.bidirectional = bidirectional
        # define the aggregation type of the features for the classifier for example, you can take the mean
        self.aggregation_type = aggregation_type
        self.encoder = nn.Linear(feat_size, emb_size, bias=True)
        # Create a (bidirectional) LSTM to encode sequence
        self.lstm = nn.LSTM(emb_size, hidden_size, batch_first=True, 
                            bidirectional=bidirectional)
        
        # The output of the LSTM doubles if we use a bidirectional encoder.
        encoding_size = hidden_size * 2 if bidirectional else hidden_size
        self.combination_layer = nn.Linear(encoding_size, encoding_size)
        
        # Create affine layer to project to the classes 
        self.projection = nn.Linear(encoding_size, tag_size)
        # dropout layer for regularizetion of a sequence
        self.dropout_layer = nn.Dropout(p=dropout)
        self.relu = nn.ReLU()
 
    def forward(self, x, seq_mask=None, seq_len=None):
        # input size
        # [B, T, feat_size] batch, time and features
        # return unormalized probabilities (logits)
        # the loss will compute the sigmoid and negative log-likelihood
        #output size
        # [B, num_class] batch, and 1 class 
        
        #[B, T, F] batch, time, features 
        h1 = self.encoder(x)
        h1 = self.relu(h1)
        # [B, T, H] batch, time, hidden or hidden * 2
        outputs, (final, _) = self.lstm(h1)
      
        if self.aggregation_type == 'mean':
            # mean over hidden states of LSTM
            outputs = self.dropout_layer(outputs)
            h = self.relu(self.combination_layer(outputs))
            #[B, H] batch, hidden
            h = h.mean(dim=1) # mean over time dimension 
        elif self.aggregation_type == 'last_state': 
            # last hidden state of the lstm or concat of bidirectional forward and backward states
            if self.bidirectional:
                h_T_fwd = final[0] # lstm 1, last hidden state of forward lstm
                h_T_bwd = final[1] # lstm 2. last hidden state of backward lstm
                #[B, H*2]
                h = torch.cat([h_T_fwd, h_T_bwd], dim=-1) # concatenate the forward with the backward in the last dimension (feat)
            else:
                h = final[-1]
            h = self.relu(self.combination_layer(h))
            h = self.dropout_layer(h)
        #[B, H] # summary for each patient
        #[B, 1]
        logits = self.projection(h)
        
        return logits

In [10]:
#eval model
def eval_model(model, dataset, device):
    model.eval()
    sigmoid = nn.Sigmoid()
    with torch.no_grad():
        y_true = []
        predictions = []
        for data, labels in dataset:
            data = data.to(device)
            labels = labels.to(device)
            logits = model(data)
            probs = sigmoid(logits) 
            predictions += [p.item() for p in probs]  # concatenate all predictions
            y_true += [y.item() for y in labels]  # concatenate all labels
    results = print_metrics_binary(y_true, predictions, logging)
    performance_with_cutoff(y_true, predictions)
    
    return results, predictions, y_true

In [11]:
def args_detail(args):
    arg_detail = "_dropout_"+str(args['dropout'])+",batch_size_"+str(args['batch_size'])+ ",lr_"+str(args['lr'])+",epochs_"+str(args['epochs'])
    return arg_detail

In [12]:
def train(args):
    mode = 'train'
    hidden_size = args['dim']
    dropout = args['dropout']
    batch_size = args['batch_size']
    learning_rate = args['lr']
    num_epochs = args['epochs']
    emb_size = args['emb_size']
    aggregation_type = args['aggregation_type']
    bidirectional_encoder = args['bidirectional'] 
    seed = args['seed']
    steps = args['steps']
    data = args['data']
    notes = args['notes']
    timestep = args['timestep']
    normalizer_state = args['normalizer_state']
    
    if seed: 
        torch.manual_seed(seed)
        np.random.seed(seed)
    device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")   
  
    logging.basicConfig(level=logging.INFO,format='%(asctime)s %(message)s',datefmt='%Y-%m-%d %H:%M:%S', )
    
    # define cvs data readers

    # if(notes != None):
    train_reader = InHospitalMortalityReader(dataset_dir=os.path.join(data, 'train'),
                                            notes_dir=notes,
                                            listfile=os.path.join(notes, 'train_listfile.csv'),
                                            period_length=48.0)

    val_reader = InHospitalMortalityReader(dataset_dir=os.path.join(data, 'train'),
                                        notes_dir=notes,
                                        listfile=os.path.join(notes, 'val_listfile.csv'),
                                        period_length=48.0)
    # else:
    #     train_reader = InHospitalMortalityReader(dataset_dir=os.path.join(data, 'train'),
    #                                          notes_dir=None,
    #                                          listfile=os.path.join(notes, 'train_listfile.csv'),
    #                                          period_length=48.0)

    #     val_reader = InHospitalMortalityReader(dataset_dir=os.path.join(data, 'train'),
    #                                         notes_dir=None,
    #                                         listfile=os.path.join(notes, 'val_listfile.csv'),
    #                                         period_length=48.0)
    

    train_dataset = MIMICDataset(train_reader, batch_labels=True)
    train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = MIMICDataset(val_reader, batch_labels=True) 
    val_dl = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    #[B, T, feat_size] batch, time, and num of variables
    feat_size = train_dataset.data.shape[-1] 
  
    # Define the classification model
    model = LSTMClassifier(tag_size=1, # binary
                      feat_size= feat_size, 
                      hidden_size=hidden_size,
                      emb_size=emb_size,
                      bidirectional=bidirectional_encoder,
                      dropout=dropout,
                      aggregation_type=aggregation_type)

    model = model.to(device)
    logging.info(args)
    logging.info(model)

    # Define optimizer
    optimizer = Adam(model.parameters(), lr=learning_rate) 
  
    # define loss for binary classification
    criterion = nn.BCEWithLogitsLoss()

    # path to save model with extension .pt on disk
    args_detail = "_dropout_"+str(args['dropout'])+",batch_size_"+str(args['batch_size'])+ ",lr_"+str(args['lr'])+",epochs_"+str(args['epochs'])
    best_val_auc = 0.

    results = []

    step = 0
    num_batches = 0
    e_losses = []

    for epoch in range(num_epochs):
        loss_batch2 = 0.0
        model.train()
        for x, labels in train_dl:
            x = x.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            y_logits = model(x)
            loss = criterion(y_logits, labels)
            loss_batch2 += loss.item() # keep track of loss 
            loss.backward()
            optimizer.step()
            step += 1
            num_batches += 1

            # Every number of steps show the model loss
            if step % steps == 0:
                logging.info("epoch (%d) step %d: training loss = %.2f"% (epoch, step, loss_batch2/num_batches))
                e_losses.append(loss_batch2/num_batches)

        metrics_results, _, _ = eval_model(model,
                                val_dl,
                                device)
        metrics_results['epoch'] = epoch
        # save results of current epoch
        results.append(metrics_results)

        print(metrics_results)

        # Model selection 
        if metrics_results['auroc'] > best_val_auc:
            best_val_auc = metrics_results['auroc']  
            model_save_name2 = './saved_models/best_LSTM_mean_bi_feat'+args_detail+'.pth'
            torch.save(model.state_dict(), model_save_name2)

    model_save_name1 = './saved_models/latest_LSTM_mean_bi_feat'+args_detail+'.pth'     
    torch.save(model.state_dict(), model_save_name1)

    plt.plot(e_losses)

In [ ]:
# execute training 
# define hyperparameters
args = {'dim': 35,
        'dropout': 0.2,
        'batch_size': 64, 
        'lr': 1e-3,
        'epochs': 20,
        'emb_size': 35, 
        'aggregation_type': 'mean',
        'bidirectional': True,
        'seed':42,
        'steps': 1000, # print loss every num of steps
        'data':'data/AKI', # path to MIMIC data
        'notes': 'data/AKI', # text not used in this model
        'timestep': 1.0, # observations every hour
        'imputation': 'previous', # imputation method
        'normalizer_state': None} # we use normalization config
train(args)

In [33]:
# for plot name
args_detail = args_detail(args)

# Test
Use the last trained model or the best validation model and run in test.
Also use calibration curves.

In [ ]:
#test

def test(args):
    # define trainning and validation datasets
    mode = 'test'
    hidden_size = args['dim']
    dropout = args['dropout']
    batch_size = args['batch_size']
    emb_size = args['emb_size']
    best_model = args['best_model']
    data = args['data']
    notes = args['notes']
    timestep = args['timestep']
    aggregation_type = args['aggregation_type']
    bidirectional_encoder = args['bidirectional']
    device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")   
    # 1. Get a unique working directory 
    
    logging.basicConfig(level=logging.INFO, 
            format='%(asctime)s %(message)s', 
            datefmt='%Y-%m-%d %H:%M:%S')
    


    test_reader = InHospitalMortalityReader(dataset_dir=os.path.join(data, 'test'),
                                           notes_dir=notes,  
                                         listfile=os.path.join(notes, 'test_listfile.csv'),
                                         period_length=48.0)

    # Read data
    test_dataset = MIMICDataset(test_reader, batch_labels=True)
    test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    #[B, M, feat_size]
    feat_size = test_dataset.data.shape[-1] 


    # Define the classification model.
    model = LSTMClassifier(tag_size=1, #binary
                    feat_size= feat_size, 
                    hidden_size=hidden_size,
                    emb_size=emb_size,
                    bidirectional=bidirectional_encoder,
                    dropout=dropout,
                    aggregation_type=aggregation_type)
    #load trained model from file
    model.load_state_dict(torch.load(best_model))
    logging.info(model)
    model = model.to(device)

    metrics_results, pred_probs, y_true = eval_model(model,
                                test_dl,
                                device)
    return metrics_results, pred_probs, y_true

In [ ]:
# Run test on saved model latest_classifier
args = {'best_model':'./saved_models/latest_LSTM_mean_bi_feat'+args_detail+'.pth',
        'dim': 35,
        'dropout': 0.2,
        'batch_size': 16,
        'emb_size': 35,
        'aggregation_type': 'mean',
        'bidirectional': True,
        'data':'data/AKI', # path to data
        'notes':'data/test', # text not used in this model
        'timestep': 1.0,
        'imputation': 'previous',
        'normalizer_state': None}
print('Result of latest_classifier')        
metrics_results, pred_probs, y_true = test(args)

In [ ]:
# Run test on saved model best_classifier
# remember to use the same hyperparameters as in training
args = {'best_model': './saved_models/best_LSTM_mean_bi_feat'+args_detail+'.pth',
        'dim': 35,
        'dropout': 0.2,
        'batch_size': 16,
        'emb_size': 35,
        'aggregation_type': 'mean',
        'bidirectional': True,
        'data': 'data/AKI', #path to data
        'notes': 'data/test', #the code ignores the text
        'timestep': 1.0,
        'imputation': 'previous',
        'normalizer_state': None}
print('Result of best_classifier')        
metrics_results, pred_probs, y_true = test(args)

# Plot

In [ ]:
# Figures ROC and calibration curve
import matplotlib.pyplot as plt
import sys
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
import pickle
from sklearn import metrics
import pandas as pd

In [ ]:
# roc curve
lstm_fpr, lstm_tpr, lstm_thresholds = metrics.roc_curve(y_true, pred_probs)

# plot the roc curve for the model
plt.figure()
plt.ylim(0., 1.0)
plt.xlim(0.,1.0)
plt.plot(lstm_fpr, lstm_tpr, marker='.', label='LSTM_feat', color='darkorange')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()

In [ ]:
# pr curve
lstm_precision, lstm_recall, _ = metrics.precision_recall_curve(y_true, pred_probs)

# plot the precision-recall curves
plt.plot([0, 1], [1, 0], linestyle='--', label='No Skill')
plt.plot(lstm_recall, lstm_precision, marker='.', label='LSTM_feat')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()
plt.show()

In [ ]:
# calibration curve
lstm_y, lstm_x = calibration_curve(y_true, pred_probs, n_bins=10)
plt.figure()
plt.ylim(0., 1.0)
plt.xlim(0.,1.0)
plt.plot(lstm_x,lstm_y, marker='^', linestyle="", markersize=7, label='LSTM_feat', color='darkorange')

plt.plot([0, 1], [0, 1], color='navy', linestyle='--')

plt.xlabel('Mean predicted value')
plt.ylabel('Fraction of positives')
plt.legend()
plt.show()